## Model for 1000 points per segment

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
# from google.colab import drive
# drive.mount('/content/gdrive')
import os

In [2]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May10rd5percent_RLV_data 1000 points.csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
cross_validation_number = int(X.shape[0]*0.1)
X_val = X_train[-cross_validation_number:]
y_val = y_train[-cross_validation_number:]
X_train = X_train[:-cross_validation_number]
y_train = y_train[:-cross_validation_number]

In [3]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [7]:
i = 7;
model = create_model2(2**i)
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss = tf.keras.losses.mean_squared_error
def lr_sch3(epoch,lr):
    if (epoch >0) &(epoch<125):
      if epoch%30==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>125):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch3,verbose = 1)
bs = 256;
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
iter1 = 500

The Number of Hidden units used is:  128


In [31]:
hist = model.fit(X_train,y_train,epochs = iter1,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/500
7658/7658 [==============================] - 49s 6ms/step - loss: 0.0056 - accuracy: 0.8753 - val_loss: 0.0042 - val_accuracy: 0.8933 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/500
7658/7658 [==============================] - 47s 6ms/step - loss: 0.0036 - accuracy: 0.8955 - val_loss: 0.0045 - val_accuracy: 0.8749 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/500
7658/7658 [==============================] - 48s 6ms/step - loss: 0.0034 - accuracy: 0.8999 - val_loss: 0.0026 - val_accuracy: 0.9109 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/500
7658/7658 [==============================] - 47s 6ms/step - loss: 0.0030 - accuracy: 0.9056 - val_loss: 0.0044 - val_accuracy: 0.8878 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/500
7658/7658 [=====================

Epoch 67/500
7658/7658 [==============================] - 48s 6ms/step - loss: 3.4528e-04 - accuracy: 0.9823 - val_loss: 2.2532e-04 - val_accuracy: 0.9896 - lr: 4.0660e-04

Epoch 68: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 68/500
7658/7658 [==============================] - 48s 6ms/step - loss: 3.4599e-04 - accuracy: 0.9825 - val_loss: 3.5292e-04 - val_accuracy: 0.9743 - lr: 4.0660e-04

Epoch 69: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 69/500
7658/7658 [==============================] - 49s 6ms/step - loss: 3.3066e-04 - accuracy: 0.9833 - val_loss: 2.2971e-04 - val_accuracy: 0.9891 - lr: 4.0660e-04

Epoch 70: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 70/500
7658/7658 [==============================] - 48s 6ms/step - loss: 3.3133e-04 - accuracy: 0.9831 - val_loss: 2.6258e-04 - val_accuracy: 0.9858 - lr: 4.0660e-04

Epoch 71: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 71/500
7658/7658 [============

Epoch 100/500
7576/7658 [============================>.] - ETA: 0s - loss: 2.3606e-04 - accuracy: 0.9872
Epoch 100: saving model to /home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/5 percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 64/cp.ckpt
7658/7658 [==============================] - 49s 6ms/step - loss: 2.3578e-04 - accuracy: 0.9872 - val_loss: 2.1428e-04 - val_accuracy: 0.9901 - lr: 2.5930e-04

Epoch 101: LearningRateScheduler setting learning rate to 0.0002593.
Epoch 101/500
7658/7658 [==============================] - 48s 6ms/step - loss: 2.3266e-04 - accuracy: 0.9874 - val_loss: 3.0252e-04 - val_accuracy: 0.9806 - lr: 2.5930e-04

Epoch 102: LearningRateScheduler setting learning rate to 0.0002593.
Epoch 102/500
7658/7658 [==============================] - 49s 6ms/step - loss: 2.3783e-04 - accuracy: 0.9871 - val_loss: 1.6372e-04 - val_accuracy: 0.9911 - lr: 2.5930e-04

Epoch 103: LearningRateScheduler setting learning rate to 0.0002593


Epoch 132: LearningRateScheduler setting learning rate to 0.0001653.
Epoch 132/500
7658/7658 [==============================] - 49s 6ms/step - loss: 1.7822e-04 - accuracy: 0.9899 - val_loss: 1.8422e-04 - val_accuracy: 0.9847 - lr: 1.6530e-04

Epoch 133: LearningRateScheduler setting learning rate to 0.0001653.
Epoch 133/500
7658/7658 [==============================] - 48s 6ms/step - loss: 1.7989e-04 - accuracy: 0.9897 - val_loss: 1.7612e-04 - val_accuracy: 0.9917 - lr: 1.6530e-04

Epoch 134: LearningRateScheduler setting learning rate to 0.0001653.
Epoch 134/500
7658/7658 [==============================] - 48s 6ms/step - loss: 1.7621e-04 - accuracy: 0.9899 - val_loss: 1.5473e-04 - val_accuracy: 0.9882 - lr: 1.6530e-04

Epoch 135: LearningRateScheduler setting learning rate to 0.0001653.
Epoch 135/500
7658/7658 [==============================] - 48s 6ms/step - loss: 1.7669e-04 - accuracy: 0.9899 - val_loss: 1.5709e-04 - val_accuracy: 0.9892 - lr: 1.6530e-04

Epoch 136: LearningRateSche

Epoch 163/500
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4887e-04 - accuracy: 0.9913 - val_loss: 1.1499e-04 - val_accuracy: 0.9929 - lr: 1.1943e-04

Epoch 164: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 164/500
7658/7658 [==============================] - 49s 6ms/step - loss: 1.5110e-04 - accuracy: 0.9911 - val_loss: 1.3276e-04 - val_accuracy: 0.9912 - lr: 1.1943e-04

Epoch 165: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 165/500
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4960e-04 - accuracy: 0.9912 - val_loss: 1.4119e-04 - val_accuracy: 0.9925 - lr: 1.1943e-04

Epoch 166: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 166/500
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4975e-04 - accuracy: 0.9911 - val_loss: 1.1238e-04 - val_accuracy: 0.9933 - lr: 1.1943e-04

Epoch 167: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 167/500
7658/7

KeyboardInterrupt: 

In [31]:
i = 7;
model = create_model2(2**i)
lr = 0.000101515;
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.mean_squared_error
stopped_itera = 181; # adding stopped iteration to the function call.
remaining_iterations = iter1 - stopped_itera
def lr_sch(epoch,lr):
    epoch = epoch+181
    print(epoch)
    if (epoch >0) &(epoch<25):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>25):
        if epoch%5==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler2 = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))

model.load_weights(checkpoint_path)
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')

The Number of Hidden units used is:  128


In [32]:
hist = model.fit(X_train,y_train,epochs = remaining_iterations,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler2,cp_callback],validation_data=(X_val, y_val))

181

Epoch 1: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 1/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4602e-04 - accuracy: 0.9913 - val_loss: 1.1358e-04 - val_accuracy: 0.9930 - lr: 1.1943e-04
182

Epoch 2: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 2/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4952e-04 - accuracy: 0.9912 - val_loss: 1.3259e-04 - val_accuracy: 0.9897 - lr: 1.1943e-04
183

Epoch 3: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 3/319
7658/7658 [==============================] - 50s 7ms/step - loss: 1.4832e-04 - accuracy: 0.9912 - val_loss: 1.6065e-04 - val_accuracy: 0.9911 - lr: 1.1943e-04
184

Epoch 4: LearningRateScheduler setting learning rate to 0.000119429.
Epoch 4/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.4847e-04 - accuracy: 0.9912 - val_loss: 1.3651e-04 - val_accuracy: 0.9924 - lr: 1.1943e-04
185

Epoch 5: Learni

Epoch 33/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.0474e-04 - accuracy: 0.9934 - val_loss: 1.0549e-04 - val_accuracy: 0.9930 - lr: 4.5043e-05
214

Epoch 34: LearningRateScheduler setting learning rate to 4.5043e-05.
Epoch 34/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.0419e-04 - accuracy: 0.9933 - val_loss: 9.2472e-05 - val_accuracy: 0.9938 - lr: 4.5043e-05
215

Epoch 35: LearningRateScheduler setting learning rate to 3.8287e-05.
Epoch 35/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.0132e-04 - accuracy: 0.9935 - val_loss: 9.2002e-05 - val_accuracy: 0.9934 - lr: 3.8287e-05
216

Epoch 36: LearningRateScheduler setting learning rate to 3.8287e-05.
Epoch 36/319
7658/7658 [==============================] - 49s 6ms/step - loss: 1.0047e-04 - accuracy: 0.9936 - val_loss: 9.2339e-05 - val_accuracy: 0.9925 - lr: 3.8287e-05
217

Epoch 37: LearningRateScheduler setting learning rate to 3.8287e-05.
Epoch 37/319
765

7658/7658 [==============================] - 49s 6ms/step - loss: 8.5855e-05 - accuracy: 0.9943 - val_loss: 9.0521e-05 - val_accuracy: 0.9940 - lr: 1.6988e-05
245

Epoch 65: LearningRateScheduler setting learning rate to 1.444e-05.
Epoch 65/319
7658/7658 [==============================] - 49s 6ms/step - loss: 8.5058e-05 - accuracy: 0.9943 - val_loss: 8.6715e-05 - val_accuracy: 0.9940 - lr: 1.4440e-05
246

Epoch 66: LearningRateScheduler setting learning rate to 1.444e-05.
Epoch 66/319
7658/7658 [==============================] - 49s 6ms/step - loss: 8.3674e-05 - accuracy: 0.9944 - val_loss: 8.6636e-05 - val_accuracy: 0.9939 - lr: 1.4440e-05
247

Epoch 67: LearningRateScheduler setting learning rate to 1.444e-05.
Epoch 67/319
7658/7658 [==============================] - 49s 6ms/step - loss: 8.3729e-05 - accuracy: 0.9944 - val_loss: 1.2539e-04 - val_accuracy: 0.9891 - lr: 1.4440e-05
248

Epoch 68: LearningRateScheduler setting learning rate to 1.444e-05.
Epoch 68/319
7658/7658 [=========

7658/7658 [==============================] - 49s 6ms/step - loss: 7.7148e-05 - accuracy: 0.9949 - val_loss: 8.0748e-05 - val_accuracy: 0.9938 - lr: 5.4460e-06
277

Epoch 97: LearningRateScheduler setting learning rate to 5.446e-06.
Epoch 97/319
7658/7658 [==============================] - 48s 6ms/step - loss: 7.7132e-05 - accuracy: 0.9949 - val_loss: 7.7421e-05 - val_accuracy: 0.9932 - lr: 5.4460e-06
278

Epoch 98: LearningRateScheduler setting learning rate to 5.446e-06.
Epoch 98/319
7658/7658 [==============================] - 50s 6ms/step - loss: 7.6901e-05 - accuracy: 0.9949 - val_loss: 7.7079e-05 - val_accuracy: 0.9939 - lr: 5.4460e-06
279

Epoch 99: LearningRateScheduler setting learning rate to 5.446e-06.
Epoch 99/319
7658/7658 [==============================] - 50s 7ms/step - loss: 7.6983e-05 - accuracy: 0.9949 - val_loss: 7.5895e-05 - val_accuracy: 0.9936 - lr: 5.4460e-06
280

Epoch 100: LearningRateScheduler setting learning rate to 4.629e-06.
Epoch 100/319
7581/7658 [=======

7658/7658 [==============================] - 49s 6ms/step - loss: 7.4122e-05 - accuracy: 0.9951 - val_loss: 7.4777e-05 - val_accuracy: 0.9935 - lr: 2.0540e-06
308

Epoch 128: LearningRateScheduler setting learning rate to 2.054e-06.
Epoch 128/319
7658/7658 [==============================] - 49s 6ms/step - loss: 7.4174e-05 - accuracy: 0.9951 - val_loss: 7.4717e-05 - val_accuracy: 0.9941 - lr: 2.0540e-06
309

Epoch 129: LearningRateScheduler setting learning rate to 2.054e-06.
Epoch 129/319
7658/7658 [==============================] - 49s 6ms/step - loss: 7.4089e-05 - accuracy: 0.9950 - val_loss: 7.4419e-05 - val_accuracy: 0.9937 - lr: 2.0540e-06
310

Epoch 130: LearningRateScheduler setting learning rate to 1.746e-06.
Epoch 130/319
7658/7658 [==============================] - 49s 6ms/step - loss: 7.3876e-05 - accuracy: 0.9951 - val_loss: 7.5002e-05 - val_accuracy: 0.9942 - lr: 1.7460e-06
311

Epoch 131: LearningRateScheduler setting learning rate to 1.746e-06.
Epoch 131/319
7658/7658 [=

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7658/7658 [==============================] - 48s 6ms/step - loss: 7.2356e-05 - accuracy: 0.9951 - val_loss: 7.3461e-05 - val_accuracy: 0.9939 - lr: 9.4000e-08
404

Epoch 224: LearningRateScheduler setting learning rate to 9.4e-08.
Epoch 224/319
7658/7658 [==============================] - 47s 6ms/step - loss: 7.2350e-05 - accuracy: 0.9951 - val_loss: 7.3480e-05 - val_accuracy: 0.9938 - lr: 9.4000e-08
405

Epoch 225: LearningRateScheduler setting learning rate to 8e-08.
Epoch 225/319
7658/7658 [==============================] - 48s 6ms/step - loss: 7.2358e-05 - accuracy: 0.9951 - val_loss: 7.3481e-05 - val_accuracy: 0.9938 - lr: 8.0000e-08
406

Epoch 226: LearningRateScheduler setting learning rate to 8e-08.
Epoch 226/319
7658/7658 [==============================] - 48s 6ms/step - loss: 7.2360e-05 - accuracy: 0.9951 - val_loss: 7.3481e-05 - val_accuracy: 0.9937 - lr: 8.0000e-08
407

Epoch 227: LearningRateScheduler setting learning rate to 8e-08.
Epoch 227/319
7658/7658 [===============

7658/7658 [==============================] - 47s 6ms/step - loss: 7.2315e-05 - accuracy: 0.9951 - val_loss: 7.3489e-05 - val_accuracy: 0.9937 - lr: 3.1000e-08
436

Epoch 256: LearningRateScheduler setting learning rate to 3.1e-08.
Epoch 256/319
7658/7658 [==============================] - 48s 6ms/step - loss: 7.2316e-05 - accuracy: 0.9951 - val_loss: 7.3462e-05 - val_accuracy: 0.9938 - lr: 3.1000e-08
437

Epoch 257: LearningRateScheduler setting learning rate to 3.1e-08.
Epoch 257/319
7658/7658 [==============================] - 47s 6ms/step - loss: 7.2313e-05 - accuracy: 0.9951 - val_loss: 7.3513e-05 - val_accuracy: 0.9937 - lr: 3.1000e-08
438

Epoch 258: LearningRateScheduler setting learning rate to 3.1e-08.
Epoch 258/319
7658/7658 [==============================] - 47s 6ms/step - loss: 7.2313e-05 - accuracy: 0.9951 - val_loss: 7.3457e-05 - val_accuracy: 0.9938 - lr: 3.1000e-08
439

Epoch 259: LearningRateScheduler setting learning rate to 3.1e-08.
Epoch 259/319
7658/7658 [=========

7658/7658 [==============================] - 47s 6ms/step - loss: 7.2300e-05 - accuracy: 0.9951 - val_loss: 7.3463e-05 - val_accuracy: 0.9938 - lr: 1.2000e-08
467

Epoch 287: LearningRateScheduler setting learning rate to 1.2e-08.
Epoch 287/319
7658/7658 [==============================] - 48s 6ms/step - loss: 7.2298e-05 - accuracy: 0.9951 - val_loss: 7.3476e-05 - val_accuracy: 0.9938 - lr: 1.2000e-08
468

Epoch 288: LearningRateScheduler setting learning rate to 1.2e-08.
Epoch 288/319
7658/7658 [==============================] - 47s 6ms/step - loss: 7.2299e-05 - accuracy: 0.9951 - val_loss: 7.3462e-05 - val_accuracy: 0.9938 - lr: 1.2000e-08
469

Epoch 289: LearningRateScheduler setting learning rate to 1.2e-08.
Epoch 289/319
7658/7658 [==============================] - 47s 6ms/step - loss: 7.2299e-05 - accuracy: 0.9951 - val_loss: 7.3455e-05 - val_accuracy: 0.9938 - lr: 1.2000e-08
470

Epoch 290: LearningRateScheduler setting learning rate to 1e-08.
Epoch 290/319
7658/7658 [===========

KeyboardInterrupt: 

# Observation that loss is constant after reaching the learning rate of <=1e-7 So its better to stop learning below this


In [33]:
model.predict(X_test)

array([[0.89360577, 0.46690145],
       [0.2609251 , 0.5638199 ],
       [0.2617634 , 0.65673465],
       ...,
       [0.25969714, 0.46219853],
       [0.065283  , 0.0539636 ],
       [0.08748144, 0.3154119 ]], dtype=float32)

In [34]:
y_test


array([[0.88940617, 0.47311156],
       [0.26437283, 0.55854691],
       [0.2419437 , 0.66788639],
       ...,
       [0.26010524, 0.46431416],
       [0.05720036, 0.06869787],
       [0.08523072, 0.31442504]])